In [ ]:
print("hello")

: 

In [ ]:
class FIRDesigner:
    def __init__(self, num_taps=64, sample_rate=48000):
        self.num_taps = num_taps
        self.sample_rate = sample_rate
        self.nyquist = sample_rate / 2
        
        # Standard graphic EQ center frequencies
        self.center_freqs = [31, 63, 125, 250, 500, 1000, 2000, 4000, 8000, 16000]
        
        self.current_taps = None
        
    def design_graphic_eq(self, gains_db):
        """
        Design FIR from graphic EQ gains
        gains_db: list of 10 dB values for each band
        """
        # Create frequency response with extra points at DC and Nyquist
        freqs = [0] + self.center_freqs + [self.nyquist]
        gains_db_full = [gains_db[0]] + list(gains_db) + [gains_db[-1]]
        
        # Convert dB to linear
        gains_linear = 10 ** (np.array(gains_db_full) / 20)
        
        # Normalize frequencies to 0-1 range
        freqs_norm = np.array(freqs) / self.nyquist
        
        # Design FIR using firwin2 (arbitrary frequency response)
        taps = signal.firwin2(self.num_taps, freqs_norm, gains_linear)
        
        self.current_taps = taps
        return taps
    
    def get_frequency_response(self, taps=None):
        """
        Calculate frequency response of FIR filter
        Returns: frequencies (Hz), magnitude (dB), phase (degrees)
        """
        if taps is None:
            taps = self.current_taps
            
        if taps is None:
            return None, None, None
            
        # Compute frequency response
        w, h = signal.freqz(taps, worN=2048, fs=self.sample_rate)
        
        # Convert to dB and degrees
        magnitude_db = 20 * np.log10(np.abs(h) + 1e-10)  # Avoid log(0)
        phase_deg = np.angle(h) * 180 / np.pi
        
        return w, magnitude_db, phase_deg
    
    def float_to_q1_15(self, taps):
        """
        Convert float taps to Q1.15 fixed-point
        Returns: list of 16-bit unsigned integers for MMIO
        """
        Q15_SCALE = 32768.0
        q15_taps = []
        
        for tap in taps:
            # Clamp to valid range
            tap_clamped = np.clip(tap, -1.0, 0.999969482421875)
            
            # Convert to Q1.15
            q15 = int(tap_clamped * Q15_SCALE)
            
            # Convert to unsigned for MMIO (two's complement)
            if q15 < 0:
                q15_unsigned = q15 + 65536
            else:
                q15_unsigned = q15
                
            q15_taps.append(q15_unsigned & 0xFFFF)
        
        return q15_taps

In [ ]:
class FIRController:
    def __init__(self, fir_mmio_base, num_taps=64):
        """
        fir_mmio_base: PYNQ MMIO object for FIR IP
        """
        self.fir = fir_mmio_base
        self.num_taps = num_taps
        
    def load_coefficients(self, coeffs):
        """
        Load Q1.15 coefficients to FPGA via MMIO
        """
        for i in range(min(len(coeffs), self.num_taps)):
            # Write to coefficient register (offset = i * 4)
            self.fir.write(i * 4, coeffs[i])
        
        # Write to control register to commit/enable filter
        # Assuming control register at offset 0x100
        self.fir.write(0x100, 1)  # Enable filter
        
        print(f"Loaded {len(coeffs)} coefficients to FPGA")
    
    def bypass(self, enable=True):
        """Enable/disable bypass mode"""
        self.fir.write(0x100, 0 if enable else 1)

In [ ]:
class GraphicEQWidget:
    def __init__(self, fir_designer, fir_controller=None):
        self.designer = fir_designer
        self.controller = fir_controller
        
        # Create sliders for 10 bands
        self.sliders = []
        slider_layout = Layout(width='60px')
        
        band_labels = ['31Hz', '63Hz', '125Hz', '250Hz', '500Hz', 
                      '1kHz', '2kHz', '4kHz', '8kHz', '16kHz']
        
        for i, label in enumerate(band_labels):
            slider = FloatSlider(
                value=0,
                min=-12,
                max=12,
                step=0.5,
                description='',
                orientation='vertical',
                readout=True,
                readout_format='.1f',
                layout=slider_layout
            )
            slider.observe(self._on_slider_change, 'value')
            self.sliders.append(slider)
        
        # Create labels below sliders
        labels = [widgets.Label(label, layout=Layout(width='60px')) 
                 for label in band_labels]
        
        # Create preset buttons
        self.btn_flat = Button(description='Flat')
        self.btn_vshape = Button(description='V-Shape')
        self.btn_bright = Button(description='Bright')
        self.btn_warm = Button(description='Warm')
        self.btn_telephone = Button(description='Telephone')
        
        self.btn_flat.on_click(lambda b: self._load_preset([0]*10))
        self.btn_vshape.on_click(lambda b: self._load_preset([6, 4, 2, 0, -3, -3, 0, 2, 4, 6]))
        self.btn_bright.on_click(lambda b: self._load_preset([0, 0, 0, 0, 2, 4, 6, 6, 6, 6]))
        self.btn_warm.on_click(lambda b: self._load_preset([3, 2, 1, 0, 0, -2, -4, -6, -6, -6]))
        self.btn_telephone.on_click(lambda b: self._load_preset([-12, -12, -6, 0, 3, 3, 0, -6, -12, -12]))
        
        # Apply to FPGA button
        self.btn_apply = Button(description='Apply to FPGA', 
                                button_style='success',
                                layout=Layout(width='150px'))
        self.btn_apply.on_click(self._on_apply)
        
        # Output widget for plots
        self.plot_output = Output()
        
        # Layout
        sliders_box = HBox(self.sliders)
        labels_box = HBox(labels)
        presets_box = HBox([self.btn_flat, self.btn_vshape, self.btn_bright, 
                           self.btn_warm, self.btn_telephone])
        
        controls_box = VBox([
            widgets.Label('Graphic EQ - 10 Band'),
            sliders_box,
            labels_box,
            widgets.Label(''),
            widgets.Label('Presets:'),
            presets_box,
            widgets.Label(''),
            self.btn_apply if self.controller else widgets.Label('(No FPGA connected)'),
        ])
        
        self.widget = HBox([controls_box, self.plot_output])
        
        # Initial plot
        self._update_plot()
    
    def _get_gains(self):
        """Get current gain values from sliders"""
        return [slider.value for slider in self.sliders]
    
    def _load_preset(self, gains):
        """Load preset values to sliders"""
        for slider, gain in zip(self.sliders, gains):
            slider.value = gain
    
    def _on_slider_change(self, change):
        """Called when any slider changes"""
        self._update_plot()
    
    def _update_plot(self):
        """Update frequency response plot"""
        gains_db = self._get_gains()
        
        # Design filter
        taps = self.designer.design_graphic_eq(gains_db)
        
        # Get frequency response
        freqs, mag_db, phase_deg = self.designer.get_frequency_response(taps)
        
        if freqs is None:
            return
        
        # Clear and redraw
        with self.plot_output:
            self.plot_output.clear_output(wait=True)
            
            fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 8))
            
            # Magnitude plot
            ax1.semilogx(freqs, mag_db, 'b-', linewidth=2)
            ax1.grid(True, which='both', alpha=0.3)
            ax1.set_xlabel('Frequency (Hz)')
            ax1.set_ylabel('Magnitude (dB)')
            ax1.set_title(f'Frequency Response - {self.designer.num_taps} Tap FIR')
            ax1.set_xlim([20, 20000])
            ax1.set_ylim([-18, 18])
            ax1.axhline(y=0, color='k', linestyle='--', alpha=0.3)
            
            # Mark center frequencies
            for freq in self.designer.center_freqs:
                ax1.axvline(x=freq, color='r', linestyle=':', alpha=0.2)
            
            # Phase plot
            ax2.semilogx(freqs, phase_deg, 'g-', linewidth=2)
            ax2.grid(True, which='both', alpha=0.3)
            ax2.set_xlabel('Frequency (Hz)')
            ax2.set_ylabel('Phase (degrees)')
            ax2.set_title('Phase Response')
            ax2.set_xlim([20, 20000])
            
            plt.tight_layout()
            plt.show()
    
    def _on_apply(self, button):
        """Apply current filter to FPGA"""
        if self.controller is None:
            print("No FPGA controller connected!")
            return
        
        gains_db = self._get_gains()
        
        # Design filter
        taps = self.designer.design_graphic_eq(gains_db)
        
        # Convert to Q1.15
        q15_taps = self.designer.float_to_q1_15(taps)
        
        # Send to FPGA
        self.controller.load_coefficients(q15_taps)
        
        print(f"✓ Applied EQ to FPGA: {gains_db}")
    
    def display(self):
        """Display the widget"""
        return self.widget